In [1]:
# Importing the required libraries
import pandas as pd # to handle the dataset
import numpy as np # to manipulate the data

import nltk
from nltk.corpus import stopwords # to remove unnecessary words
from nltk.stem.porter import PorterStemmer # to get the stem of the words

import re # regular expression to clean the data

from tensorflow.keras.layers import Embedding #to create the Embedding matrix
from tensorflow.keras.layers import LSTM # predicting using the Long Short Term Memory model
from tensorflow.keras.layers import Dropout #for the regularization(handling the overfitting)
from tensorflow.keras.layers import Dense #to create the output layer
from tensorflow.keras.models import Sequential # for creating the sequential model
from tensorflow.keras.preprocessing.text import one_hot # to create the one_hot representation of the words
from tensorflow.keras.preprocessing.sequence import pad_sequences # inorder to make all the sentences of the same length by adding some padding to them

from sklearn.model_selection import train_test_split # to split the dataset into the train and test data
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report #performance metrics to observe the performance of the model

### Dataset

> The dataset has been downloaded from the kaggle for this project. The link for the dataset is as follow:-
    - https://www.kaggle.com/samrat96/fake-news-detection?select=train.csv

> I will be using only the train.csv for the practise. So I will be spliting the data into the train and test for analysing the performance of the model

In [2]:
#Importing the dataset
df = pd.read_csv('train.csv')

In [3]:
# observing the top 5 dataset
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


> Here label represents the dependent or the output variable and other are the independent features. I will be using the column title for this prediction

In [4]:
#Checking the null value
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

since there is only 39 missing values also it is difficult to fill the missing values in the text data so I am going to simply drop those columns

In [5]:
#Dropping the row with the null columns
df = df.dropna()

In [6]:
# we need to reset the index so that we can loop through all the data easily later on 
df.reset_index(inplace = True)

In [7]:
#assigning the text to new variable for the easy understanding and manipulation
news = df['title']

In [8]:
#printing the top 5 news
news.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

### Data Preprocessing

In [10]:
# function to remove all the numbers, punctuation, lowering all the alphabets in the news
corpus = []
ps = PorterStemmer()
for i in range(len(news)):
    print(i)
    sentence_with_only_aplha = re.sub('[^a-zA-Z]', ' ', news[i]) #removing everything else the alphabet from the sentence
    sentence_lower_alpha = sentence_with_only_aplha.lower() # conveting all the alphabet in the into the lowercase
    word_list = sentence_lower_alpha.split() # splitting the dataset into words
    stem_word_list = [ps.stem(word) for word in word_list if word not in set(stopwords.words('english'))] # removing the stopwords(unimportant words from the word list or from the news and finding the stem or base of the word and creating the list of only the stem words)
    clean_sentence = ' '.join(stem_word_list) # joining the stem words to form the sentence again
    corpus.append(clean_sentence) # creating the list with the clean sentences

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068


3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726


5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403


6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066


...

11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
11580
11581
11582
11583
11584
11585
11586
11587
11588
11589
11590
11591
11592
11593
11594
11595
11596
11597
11598
11599
11600
11601
11602
11603
11604
11605
11606
11607
11608
11609
11610
11611
11612
11613
11614
11615
11616
11617
11618
11619
11620
11621
11622
11623
11624
11625
11626
11627
11628
11629
11630
11631
11632
11633
11634
11635
11636
11637
11638
11639
11640
11641
11642
11643
11644
11645
11646
11647
11648
11649
11650
11651
11652
11653
11654
11655
11656
11657
11658
11659
11660
11661
11662
11663
11664
11665
11666
11667
11668
11669
11670
11671
11672
11673
11674
11675
11676
11677
1167

12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
12948
12949
12950
12951
12952
12953
12954
12955
12956
12957
12958
12959
12960
12961
12962
12963
12964
12965
12966
12967
12968
12969
12970
12971
12972
12973
12974
12975
12976
12977
12978
12979
12980
12981
12982
12983
12984
12985
12986
12987
12988
12989
12990
12991
12992
12993
12994
12995
12996
12997
12998
12999
13000
13001
13002
13003
13004
13005
13006
13007
13008
13009
13010
13011
13012
13013
13014
13015
13016
13017
13018
13019
13020
13021
13022
13023
13024
13025
13026
13027
13028
13029
13030
13031
13032
13033
13034
13035
13036
13037
13038
13039
13040
13041
13042
13043
13044
13045
13046
13047
13048
13049
13050
13051
13052
13053
13054
13055
13056
13057
13058
13059
13060
13061
13062
13063
13064
13065
13066
13067
13068
13069
13070
13071
13072
13073
13074
13075
13076
13077
13078
13079
13080
13081
13082
1308

14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
14311
14312
14313
14314
14315
14316
14317
14318
14319
14320
14321
14322
14323
14324
14325
14326
14327
14328
14329
14330
14331
14332
14333
14334
14335
14336
14337
14338
14339
14340
14341
14342
14343
14344
14345
14346
14347
14348
14349
14350
14351
14352
14353
14354
14355
14356
14357
14358
14359
14360
14361
14362
14363
14364
14365
14366
14367
14368
14369
14370
14371
14372
14373
14374
14375
14376
14377
14378
14379
14380
14381
14382
14383
14384
14385
14386
14387
14388
14389
14390
14391
14392
14393
14394
14395
14396
14397
14398
14399
14400
14401
14402
14403
14404
14405
14406
14407
14408
14409
14410
14411
14412
14413
14414
14415
14416
14417
14418
14419
14420
14421
14422
14423
14424
14425
14426
14427
14428
14429
14430
14431
14432
14433
14434
14435
14436
14437
14438
14439
14440
14441
14442
14443
14444
14445
14446
14447
14448
14449
14450
14451
14452
14453
14454
14455
14456
14457
1445

15713
15714
15715
15716
15717
15718
15719
15720
15721
15722
15723
15724
15725
15726
15727
15728
15729
15730
15731
15732
15733
15734
15735
15736
15737
15738
15739
15740
15741
15742
15743
15744
15745
15746
15747
15748
15749
15750
15751
15752
15753
15754
15755
15756
15757
15758
15759
15760
15761
15762
15763
15764
15765
15766
15767
15768
15769
15770
15771
15772
15773
15774
15775
15776
15777
15778
15779
15780
15781
15782
15783
15784
15785
15786
15787
15788
15789
15790
15791
15792
15793
15794
15795
15796
15797
15798
15799
15800
15801
15802
15803
15804
15805
15806
15807
15808
15809
15810
15811
15812
15813
15814
15815
15816
15817
15818
15819
15820
15821
15822
15823
15824
15825
15826
15827
15828
15829
15830
15831
15832
15833
15834
15835
15836
15837
15838
15839
15840
15841
15842
15843
15844
15845
15846
15847
15848
15849
15850
15851
15852
15853
15854
15855
15856
15857
15858
15859
15860
15861
15862
15863
15864
15865
15866
15867
15868
15869
15870
15871
15872
15873
15874
15875
15876
15877
15878
1587

17100
17101
17102
17103
17104
17105
17106
17107
17108
17109
17110
17111
17112
17113
17114
17115
17116
17117
17118
17119
17120
17121
17122
17123
17124
17125
17126
17127
17128
17129
17130
17131
17132
17133
17134
17135
17136
17137
17138
17139
17140
17141
17142
17143
17144
17145
17146
17147
17148
17149
17150
17151
17152
17153
17154
17155
17156
17157
17158
17159
17160
17161
17162
17163
17164
17165
17166
17167
17168
17169
17170
17171
17172
17173
17174
17175
17176
17177
17178
17179
17180
17181
17182
17183
17184
17185
17186
17187
17188
17189
17190
17191
17192
17193
17194
17195
17196
17197
17198
17199
17200
17201
17202
17203
17204
17205
17206
17207
17208
17209
17210
17211
17212
17213
17214
17215
17216
17217
17218
17219
17220
17221
17222
17223
17224
17225
17226
17227
17228
17229
17230
17231
17232
17233
17234
17235
17236
17237
17238
17239
17240
17241
17242
17243
17244
17245
17246
17247
17248
17249
17250
17251
17252
17253
17254
17255
17256
17257
17258
17259
17260
17261
17262
17263
17264
17265
1726

In [12]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [13]:
# now creating the one hot representation of the dataset
voc_size = 5000 # defining the dictionary with the 5000 words
one_hot_representation = [one_hot(sentence,voc_size) for sentence in news] # converting the dataset into one hot representation
print(one_hot_representation)

[[3200, 2538, 3852, 252, 852, 3376, 3474, 2230, 2509, 529, 4244, 1981, 2771, 966], [904, 4701, 4371, 952, 2579, 3031, 2194, 4285], [4049, 2968, 2957, 522, 4279, 4904, 3978], [1219, 4102, 3080, 1929, 1479, 4350, 3874, 1889, 84, 3154], [4536, 2579, 2418, 970, 1986, 4744, 4568, 658, 2579, 985, 2507, 2644, 970, 4465], [1218, 1748, 3534, 4726, 3834, 1185, 1471, 175, 2608, 3643, 314, 890, 805, 2781, 4837, 3618, 4349, 3945, 4285], [1550, 2939, 2923, 1075, 278, 4902, 1554, 2865, 2968, 163, 3165, 4533], [3331, 4922, 3864, 1224, 970, 4734, 4363, 3662, 3658, 2781, 1185, 1285, 2968, 163, 3165, 4533], [1176, 2747, 970, 2978, 4939, 4871, 2328, 1679, 444, 2507, 157, 2811, 3109], [1785, 4345, 2541, 168, 3244, 2781, 4407, 897, 411], [3273, 229, 3411, 916, 3437, 28, 1545, 2286, 3473, 1929, 2968, 1417, 2253], [4350, 4012, 3474, 3202, 1686, 2771, 1185, 4363, 3662], [516, 4621, 4494, 2895, 2000, 2660, 717, 3031, 2300, 3290, 662, 405, 4363, 173], [1929, 1490, 3299, 1627, 570, 1979, 3331, 1475, 4363, 700, 29

### Word Embedding representation

In [14]:
# we need to define the sentence length for padding so I need to find the length of the longest sentence
sent_length = max([len(sentence) for sentence in corpus])

In [15]:
%%time
# creating the embeded docs from the onehot representation which will be converted to the embedded matrix later using Embeded layer of the keras
embedding_doc = pad_sequences(one_hot_representation, padding = 'pre', maxlen= sent_length)
embedding_doc

Wall time: 66.3 ms


array([[   0,    0,    0, ..., 1981, 2771,  966],
       [   0,    0,    0, ..., 3031, 2194, 4285],
       [   0,    0,    0, ..., 4279, 4904, 3978],
       ...,
       [   0,    0,    0, ...,  163, 3165, 4533],
       [   0,    0,    0, ..., 1530, 1929, 4444],
       [   0,    0,    0, ..., 4219, 2446, 1277]])

### Creating the model

In [16]:
%%time
feature_dim = 50 # this will create the embeded matrix of the dimention 50 * number of words(5000)
model = Sequential() # initialising the sequential model
model.add(Embedding(voc_size, feature_dim, input_length = sent_length)) # creating the Embedded matrix
model.add(Dropout(0.3)) # dropping some of the neurons to handle the overfitting
model.add(LSTM(100)) # creating the LSTM with the 100 neurons
model.add(Dropout(0.3)) # handling the overfitting by dropping the 30% neurons 
model.add(Dense(1, activation = 'sigmoid')) # output layers with two output
model.compile(loss = 'binary_crossentropy', metrics = ['mse','accuracy'], optimizer = 'adam') # compiling the model with the loss function of binary_crossentropy because we have only two output feature, and using different metrics for the performance and adam for the optimization
print(model.summary()) # printing the summary of the model(what is the model made up of)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 299, 50)           250000    
_________________________________________________________________
dropout (Dropout)            (None, 299, 50)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None
Wall time: 833 ms


In [17]:
print(len(embedding_doc))
print(df['label'].shape) #checking the shape of enbedded_docs and the output feature

18285
(18285,)


In [18]:
# converting the data into the array for splitting the data into the train and test and giving input the Network
X = np.array(embedding_doc)
y = np.array(df['label'])

In [19]:
#checking the shape of the dependent and indepandent feature
X.shape, y.shape

((18285, 299), (18285,))

In [20]:
# spliting the dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state= 42)

### Training the model

In [21]:
X_train.shape

(12250, 299)

In [22]:
%%time
model.fit(X_train,y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64, verbose=1)

Epoch 1/10
192/192 [==============================] - 54s 282ms/step - loss: 0.3356 - mse: 0.1091 - accuracy: 0.8436 - val_loss: 0.1972 - val_mse: 0.0604 - val_accuracy: 0.9176
Epoch 2/10
192/192 [==============================] - 60s 314ms/step - loss: 0.1338 - mse: 0.0390 - accuracy: 0.9473 - val_loss: 0.1912 - val_mse: 0.0583 - val_accuracy: 0.9205
Epoch 3/10
192/192 [==============================] - 61s 320ms/step - loss: 0.0810 - mse: 0.0226 - accuracy: 0.9706 - val_loss: 0.2118 - val_mse: 0.0607 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 59s 309ms/step - loss: 0.0489 - mse: 0.0128 - accuracy: 0.9836 - val_loss: 0.2690 - val_mse: 0.0640 - val_accuracy: 0.9218
Epoch 5/10
192/192 [==============================] - 60s 313ms/step - loss: 0.0278 - mse: 0.0070 - accuracy: 0.9911 - val_loss: 0.3130 - val_mse: 0.0722 - val_accuracy: 0.9125
Epoch 6/10
192/192 [==============================] - 60s 312ms/step - loss: 0.0173 - mse: 0.0043 - accuracy: 0.994

### Performace of the model

In [28]:
y_pred = model.predict_classes(X_test) # predicting the test dataset

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [29]:
y_test

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [30]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [31]:
# printing the confusion matrix of the dataset
confusion_matrix(y_test, y_pred)

array([[3092,  327],
       [ 181, 2435]], dtype=int64)

In [32]:
#printing the accuracy of the model
accuracy_score(y_test, y_pred)

0.915824357912179

In [34]:
# printing the classification report of the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      3419
           1       0.88      0.93      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.92      0.92      6035

